In [459]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras_preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [460]:
data = pd.read_csv('D:\Descargas\IMDB Dataset.csv')

data['review'] = data['review'].str.lower()


nuevo:

In [461]:
prueba = data.copy()

In [462]:
from nltk.corpus import wordnet

In [463]:
def Negation(sentence):	
  temp = int(0)
  for i in range(len(sentence)):
      if sentence[i-1] in ['not',"n't"]:
          antonyms = []
          for syn in wordnet.synsets(sentence[i]):
              syns = wordnet.synsets(sentence[i])
              w1 = syns[0].name()
              temp = 0
              for l in syn.lemmas():
                  if l.antonyms():
                      antonyms.append(l.antonyms()[0].name())
              max_dissimilarity = 0
              for ant in antonyms:
                  syns = wordnet.synsets(ant)
                  w2 = syns[0].name()
                  syns = wordnet.synsets(sentence[i])
                  w1 = syns[0].name()
                  word1 = wordnet.synset(w1)
                  word2 = wordnet.synset(w2)
                  if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                      temp = 1 - word1.wup_similarity(word2)
                  if temp>max_dissimilarity:
                      max_dissimilarity = temp
                      antonym_max = ant
                      sentence[i] = antonym_max
                      sentence[i-1] = ''
  while '' in sentence:
      sentence.remove('')
  return sentence


In [464]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


ejemplo:

In [465]:
sentence = word_tokenize("i didn't like you")
#print(word_tokenize("he doesn't like you"))
print(Negation(sentence))

['i', 'did', 'dislike', 'you']


fin ejemplo:

In [466]:
prueba.head(10)

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
5,"probably my all-time favorite movie, a story o...",positive
6,i sure would like to see a resurrection of a u...,positive
7,"this show was an amazing, fresh & innovative i...",negative
8,encouraged by the positive comments about this...,negative
9,if you like original gut wrenching laughter yo...,positive


In [467]:
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result

In [468]:
sincaracteres = prueba["review"].apply(lambda cw : remove_tags(cw))

In [469]:
sincaracteres2 = []
for i in range(len(sincaracteres)):
    negado = []
    for j in range(len(sincaracteres[i].split("."))):
        sent = Negation(word_tokenize(sincaracteres[i].split(".")[j]))
        negado.append(sent)
    sincaracteres2.append(negado)

In [470]:
reviews2 = []
for i in range(len(sincaracteres2)):
    lista = []
    for j in range(len(sincaracteres2[i])):
        nueva = ' '.join(sincaracteres2[i][j])
        lista.append(nueva)
    lista2 = '.'.join(lista)
    reviews2.append(lista2)

In [471]:
reviewsf = []
for i in range(len(reviews2)):
    reviews2[i] = reviews2[i].replace(" '", "'")
    reviews2[i] = reviews2[i].replace(" ,", ",")
    reviews2[i] = reviews2[i].replace(" n't", "n't")
    reviewsf.append(reviews2[i])

In [472]:
prueba["review"] = reviewsf

viejo:

In [473]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [474]:
def remove_stopwords(data):
  data['review without stopwords'] = data['review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

#def remove_tags(string):
#    result = re.sub('<.*?>','',string)
#    return result


In [475]:
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')

C:\Users\danie\AppData\Local\Temp\ipykernel_25428\2824813740.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')


bloque nuevo:

In [476]:
data_without_stopwords2 = remove_stopwords(prueba)
data_without_stopwords2['clean_review'] = data_without_stopwords2['review without stopwords']
data_without_stopwords2['clean_review'] = data_without_stopwords2['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')

C:\Users\danie\AppData\Local\Temp\ipykernel_25428\2951378201.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data_without_stopwords2['clean_review'] = data_without_stopwords2['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')


In [477]:
data_without_stopwords.head()

,review,sentiment,review without stopwords,clean_review
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...


In [478]:
reviews = data_without_stopwords['clean_review']
reviews

0        one reviewers mentioned watching just 1 oz epi...
1        wonderful little production  the filming techn...
2        thought wonderful way spend time hot summer we...
3        basically family little boy  jake  thinks zomb...
4        petter mattei s  love time money  visually stu...
                               ...                        
49995    thought movie right good job  wasn t creative ...
49996    bad plot  bad dialogue  bad acting  idiotic di...
49997    catholic taught parochial elementary schools n...
49998    going disagree previous comment side maltin on...
49999    no one expects star trek movies high art  fans...
Name: clean_review, Length: 50000, dtype: object

bloque nuevo:

In [479]:
reviewsneg = data_without_stopwords2['clean_review']
reviewsneg

0        one reviewers mentioned watching just 1 oz epi...
1        wonderful little production the filming techni...
2        thought wonderful way spend time hot summer we...
3        basically family little boy   jake   thinks zo...
4        petter mattei s    love time money   visually ...
                               ...                        
49995    thought movie right good job it wasn t creativ...
49996    bad plot  bad dialogue  bad acting  idiotic di...
49997    catholic taught parochial elementary schools n...
49998    going disagree previous comment side maltin on...
49999    no one expects star trek movies high art  fans...
Name: clean_review, Length: 50000, dtype: object

In [480]:
reviews_list = []
for i in range(len(reviews)):
  reviews_list.append(reviews[i])


bloque nuevo:

In [481]:
reviews_list2 = []
for i in range(len(reviewsneg)):
  reviews_list2.append(reviewsneg[i])

In [482]:
sentiment = data_without_stopwords['sentiment']

In [483]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))

In [484]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [485]:
X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)

bloque nuevo:

In [486]:
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(reviews_list2, y, test_size=0.2, random_state = 45)

In [487]:
len(Y_train)

40000

In [488]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

bloque nuevo:

In [489]:
tokenizer2 = Tokenizer(num_words=5000)
tokenizer2.fit_on_texts(X_train2)

In [490]:
words_to_index = tokenizer.word_index

bloque nuevo:

In [491]:
words_to_index2 = tokenizer2.word_index

In [492]:
len(words_to_index)

95419

In [493]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map
    

In [494]:
word_to_vec_map = read_glove_vector('D:/Descargas/glove.6B.50d.txt')

In [495]:
maxLen = 150


In [496]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

bloque nuevo:

In [497]:
vocab_len2 = len(words_to_index2)
embed_vector_len2 = word_to_vec_map['moon'].shape[0]

emb_matrix2 = np.zeros((vocab_len2, embed_vector_len2))

for word, index in words_to_index2.items():
  embedding_vector2 = word_to_vec_map.get(word)
  if embedding_vector2 is not None:
    emb_matrix2[index, :] = embedding_vector2

embedding_layer2 = Embedding(input_dim=vocab_len2, output_dim=embed_vector_len2, input_length=maxLen, weights = [emb_matrix2], trainable=False)

In [498]:
# def imdb_rating(input_shape):

#   X_indices = Input(input_shape)

#   embeddings = embedding_layer(X_indices)

#   X = LSTM(128, return_sequences=True)(embeddings)

#   X = Dropout(0.6)(X)

#   X = LSTM(128, return_sequences=True)(X)

#   X = Dropout(0.6)(X)

#   X = LSTM(128)(X)

#   X = Dense(1, activation='sigmoid')(X)

#   model = Model(inputs=X_indices, outputs=X)

#   return model

In [499]:
def conv1d_model(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = Conv1D(512,3,activation='relu')(embeddings)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  X = Dropout(0.8)(X)
  X = MaxPooling1D(3)(X)

  X = GlobalMaxPooling1D()(X)

  X = Dense(256, activation='relu')(X)
  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

bloque nuevo:

In [500]:
def conv1d_model2(input_shape):

  X_indices2 = Input(input_shape)

  embeddings2 = embedding_layer2(X_indices2)

  X2 = Conv1D(512,3,activation='relu')(embeddings2)
  
  X2 = MaxPooling1D(3)(X2)

  X2 = Conv1D(256,3,activation='relu')(X2)
  
  X2 = MaxPooling1D(3)(X2)

  X2 = Conv1D(256,3,activation='relu')(X2)
  X2 = Dropout(0.8)(X2)
  X2 = MaxPooling1D(3)(X2)

  X2 = GlobalMaxPooling1D()(X2)

  X2 = Dense(256, activation='relu')(X2)
  X2 = Dense(1, activation='sigmoid')(X2)

  model2 = Model(inputs=X_indices2, outputs=X2)

  return model2

In [501]:
# model = imdb_rating((maxLen,))
# model.summary()

In [502]:
model_1d = conv1d_model((maxLen,))
model_1d.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 150)]             0         
                                                                 
 embedding_10 (Embedding)    (None, 150, 50)           4770950   
                                                                 
 conv1d_24 (Conv1D)          (None, 148, 512)          77312     
                                                                 
 max_pooling1d_24 (MaxPoolin  (None, 49, 512)          0         
 g1D)                                                            
                                                                 
 conv1d_25 (Conv1D)          (None, 47, 256)           393472    
                                                                 
 max_pooling1d_25 (MaxPoolin  (None, 15, 256)          0         
 g1D)                                                      

bloque nuevo:

In [503]:
model_1d2 = conv1d_model2((maxLen,))
model_1d2.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 150)]             0         
                                                                 
 embedding_11 (Embedding)    (None, 150, 50)           4769850   
                                                                 
 conv1d_27 (Conv1D)          (None, 148, 512)          77312     
                                                                 
 max_pooling1d_27 (MaxPoolin  (None, 49, 512)          0         
 g1D)                                                            
                                                                 
 conv1d_28 (Conv1D)          (None, 47, 256)           393472    
                                                                 
 max_pooling1d_28 (MaxPoolin  (None, 15, 256)          0         
 g1D)                                                     

In [504]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

bloque nuevo:

In [505]:
X_train_indices2 = tokenizer2.texts_to_sequences(X_train2)

In [506]:
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
X_train_indices.shape

(40000, 150)

bloque nuevo:

In [507]:
X_train_indices2 = pad_sequences(X_train_indices2, maxlen=maxLen, padding='post')
X_train_indices2.shape

(40000, 150)

In [508]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)
model_1d.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [509]:
model_1d.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
625/625 [==============================] - 62s 98ms/step - loss: 0.5697 - accuracy: 0.6985
Epoch 2/15
625/625 [==============================] - 70s 113ms/step - loss: 0.4646 - accuracy: 0.7818
Epoch 3/15
625/625 [==============================] - 67s 107ms/step - loss: 0.4164 - accuracy: 0.8099
Epoch 4/15
625/625 [==============================] - 63s 100ms/step - loss: 0.3836 - accuracy: 0.8284
Epoch 5/15
625/625 [==============================] - 75s 119ms/step - loss: 0.3600 - accuracy: 0.8396
Epoch 6/15
625/625 [==============================] - 77s 124ms/step - loss: 0.3297 - accuracy: 0.8539
Epoch 7/15
625/625 [==============================] - 83s 133ms/step - loss: 0.3076 - accuracy: 0.8675
Epoch 8/15
625/625 [==============================] - 71s 114ms/step - loss: 0.2853 - accuracy: 0.8790
Epoch 9/15
625/625 [==============================] - 70s 112ms/step - loss: 0.2631 - accuracy: 0.8906
Epoch 10/15
625/625 [==============================] - 72s 115ms/step - lo

bloque nuevo:

In [510]:
adam2 = keras.optimizers.Adam(learning_rate = 0.001)
model_1d2.compile(optimizer=adam2, loss='binary_crossentropy', metrics=['accuracy'])
model_1d2.fit(X_train_indices2, Y_train2, batch_size=64, epochs=15)

Epoch 1/15
625/625 [==============================] - 64s 102ms/step - loss: 0.4960 - accuracy: 0.7501
Epoch 2/15
625/625 [==============================] - 66s 106ms/step - loss: 0.3671 - accuracy: 0.8353
Epoch 3/15
625/625 [==============================] - 108s 173ms/step - loss: 0.3262 - accuracy: 0.8581
Epoch 4/15
625/625 [==============================] - 95s 153ms/step - loss: 0.2805 - accuracy: 0.8812
Epoch 5/15
625/625 [==============================] - 87s 139ms/step - loss: 0.2349 - accuracy: 0.9025
Epoch 6/15
625/625 [==============================] - 74s 119ms/step - loss: 0.1963 - accuracy: 0.9202
Epoch 7/15
625/625 [==============================] - 74s 118ms/step - loss: 0.1501 - accuracy: 0.9407
Epoch 8/15
625/625 [==============================] - 78s 124ms/step - loss: 0.1154 - accuracy: 0.9560
Epoch 9/15
625/625 [==============================] - 75s 120ms/step - loss: 0.0939 - accuracy: 0.9637
Epoch 10/15
625/625 [==============================] - 70s 112ms/step - 

In [511]:
# adam = keras.optimizers.Adam(learning_rate = 0.0001)
# model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [512]:
# model.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

In [513]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

bloque nuevo:

In [514]:
X_test_indices2 = tokenizer2.texts_to_sequences(X_test2)

X_test_indices2 = pad_sequences(X_test_indices2, maxlen=maxLen, padding='post')

In [515]:
# model.evaluate(X_test_indices, Y_test)

In [536]:
model_1d.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 6s 21ms/step - loss: 0.3733 - accuracy: 0.8308


[0.3732887804508209, 0.8307999968528748]

bloque nuevo:

In [537]:
model_1d2.evaluate(X_test_indices2, Y_test2)

313/313 [==============================] - 6s 21ms/step - loss: 0.3975 - accuracy: 0.8319


[0.3974872827529907, 0.8319000005722046]

In [518]:
preds = model_1d.predict(X_test_indices)

313/313 [==============================] - 7s 23ms/step


bloque nuevo:

In [519]:
preds2 = model_1d2.predict(X_test_indices2)

313/313 [==============================] - 8s 26ms/step


In [520]:
# n = np.random.randint(0,9999)

# X_test[n]


In [521]:
# if preds[n] > 0.5:
#   print('predicted sentiment : positive')
# else: 
#   print('precicted sentiment : negative')

# if (Y_test[n] == 1):
#   print('correct sentiment : positive')
# else:
#   print('correct sentiment : negative')


In [522]:
# preds[n]

In [523]:
# Y_test[n]

In [524]:
model_1d.save_weights('D:/Descargas/imdb_weights_con1vd.hdf5')

bloque nuevo:

In [525]:
model_1d2.save_weights('D:/Descargas/imdb_weights_con1vd.hdf5_2')

In [526]:
reviews_list_idx = tokenizer.texts_to_sequences(reviews_list)

bloque nuevo:

In [527]:
reviews_list_idx2 = tokenizer2.texts_to_sequences(reviews_list2)

In [528]:
def add_score_predictions(data, reviews_list_idx):

  data['sentiment score'] = 0

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds = model_1d.predict(reviews_list_idx)

  data['sentiment score'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  data['predicted sentiment'] = 0

  data['predicted sentiment'] = pred_sentiment

  return data

Bloque nuevo:

In [529]:
def add_score_predictions2(data, reviews_list_idx):

  data['sentiment score'] = 0

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds2 = model_1d2.predict(reviews_list_idx)

  data['sentiment score'] = review_preds2

  pred_sentiment2 = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds2)))

  data['predicted sentiment'] = 0

  data['predicted sentiment'] = pred_sentiment2

  return data

In [530]:
dataf1 = data.copy()
dataf2 = data.copy()

In [531]:
dataf1 = add_score_predictions(dataf1, reviews_list_idx)

1563/1563 [==============================] - 37s 24ms/step


bloque nuevo:

In [532]:
dataf2 = add_score_predictions2(dataf2, reviews_list_idx2)

1563/1563 [==============================] - 38s 24ms/step


In [533]:
dataf1

,review,sentiment,review without stopwords,clean_review,sentiment score,predicted sentiment
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...,0.816103,positive
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...,0.982594,positive
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...,0.981700,positive
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...,0.557990,positive
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...,0.978383,positive
...,...,...,...,...,...,...
49995,i thought this movie did a down right good job...,positive,thought movie right good job. wasn't creative ...,thought movie right good job wasn t creative ...,0.973569,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative,"bad plot, bad dialogue, bad acting, idiotic di...",bad plot bad dialogue bad acting idiotic di...,0.006047,negative
49997,i am a catholic taught in parochial elementary...,negative,catholic taught parochial elementary schools n...,catholic taught parochial elementary schools n...,0.255125,negative
49998,i'm going to have to disagree with the previou...,negative,going disagree previous comment side maltin on...,going disagree previous comment side maltin on...,0.224971,negative


In [539]:
count = 0
for i in range(len(dataf1)):
    if dataf1.at[i, "sentiment"] == dataf1.at[i, "predicted sentiment"]:
        count += 1
print(count/len(dataf1)*100, "%")



95.204 %


bloque nuevo:

In [535]:
count = 0
for i in range(len(dataf2)):
    if dataf2.at[i, "sentiment"] == dataf2.at[i, "predicted sentiment"]:
        count += 1
print(count/len(dataf2)*100, "%")

95.828 %
